This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN, SuperGAT
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks.contrastive_based import *


In [2]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 50,
    'lr' : 0.01,
    'lambda' : 1
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    'alpha' : 0.15
}

generator_config = {
    'num_clusters' : 7,
}

pretext_task = SelfGNNPPR

In [3]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]

In [4]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_task=pretext_task)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)


([1.9559522867202759,
  1.9413615465164185,
  1.9273747205734253,
  1.9021227359771729,
  1.880773663520813,
  1.8432104587554932,
  1.8234498500823975,
  1.7274941205978394,
  1.675790786743164,
  1.6333739757537842,
  1.533294677734375,
  1.4951980113983154,
  1.4106639623641968,
  1.4075807332992554,
  1.2538739442825317,
  1.0828132629394531,
  1.068031907081604,
  1.081271767616272,
  0.9871228933334351,
  0.9852614402770996,
  0.9110754728317261,
  0.8105113506317139,
  0.6694647669792175,
  0.8084251284599304,
  0.7721043825149536,
  0.755142867565155,
  0.5906595587730408,
  0.6583021879196167,
  0.5190403461456299,
  0.6367490291595459,
  0.5755116939544678,
  0.5187165141105652,
  0.5578628182411194,
  0.5001803040504456,
  0.44542211294174194,
  0.4622970521450043,
  0.4742157459259033,
  0.40356969833374023,
  0.4278590977191925,
  0.4216104745864868,
  0.4633737802505493,
  0.3751392364501953,
  0.4379461705684662,
  0.4109773635864258,
  0.3207378387451172,
  0.2673716247